In [1]:
working_directory = '/users/ironstein/documents/projects working directory/handwriting recognition using neural networks on FPGA final year project/image processing/ipython notebooks/python C extension testing/mark3'

import os
import time
import graphlab as gl

os.chdir(working_directory)

In [13]:
%%file test.c

#include <stdio.h>

void for_loop_test_without_register_values(int n) {
    int i;
    int j;
    for(i=0;i<n;i++);
}

void for_loop_test_with_register_values(int n) {
    
    int var = n;
    int i;
    int p;     
    
    for(i=0;i<var;i++){
        for(p=0;p<10000;p++);
    }
}


Overwriting test.c


###generating shared object for the above C file (test.so)


In [14]:
os.system('cd ' + working_directory)
!clang -c test.c
!clang -o test.so -shared test.c
!file test.so

test.so: Mach-O 64-bit dynamically linked shared library x86_64


###python file to access the functions in test.so shared library

In [1]:
%%file _test.py

import numpy
import ctypes

_test_ = numpy.ctypeslib.load_library('test','.')

#specifying the required argument types for each function
_test_.for_loop_test_without_register_values.argtypes = [ctypes.c_int]
_test_.for_loop_test_without_register_values.restype = ctypes.c_void_p
_test_.for_loop_test_with_register_values.argtypes = [ctypes.c_int]
_test_.for_loop_test_with_register_values.restype = ctypes.c_void_p

def C_for_test_without_register_values(n) : 
    _test_.for_loop_test_without_register_values(n)
    
def C_for_test_with_register_values(n) : 
    _test_.for_loop_test_with_register_values(n)

Overwriting _test.py


In [16]:
def Python_for_test(n) : 
    for i in range(n) : 
        pass

###decorator for calculating time of execution of a function

In [17]:
def time_this_function(function) : 
    try : 
        time
    except : 
        import time
    
    def wrapper_function(*args,**kwargs) : 
        start_time = time.time()
        function(*args,**kwargs)
        elapsed_time = (time.time() - start_time)*1000
#         print(elapsed_time,end=' ms\n')
        return(elapsed_time)
        
    return wrapper_function

###testing Python and C functions for comparing the performance

In [18]:
import _test

@time_this_function
def C_test_without_register_values(n) : 
    _test.C_for_test_without_register_values(n)

@time_this_function
def C_test_with_register_values(n) : 
    _test.C_for_test_with_register_values
    
@time_this_function
def Python_test(n) : 
    Python_for_test(n)
    
# a = C_test_with_register_values(1000)
# a

###time to compare the difference

In [19]:
C_time_array_without_register_values = []
C_time_array_with_register_values = []
Python_time_array = []
count_array = []
i = 1

while(i <= 100000000) : 
    print(i)
    C_time_array_without_register_values.append(C_test_without_register_values(i))
    C_time_array_with_register_values.append(C_test_with_register_values(i))
    Python_time_array.append(Python_test(i))
    count_array.append(i)
    i *= 10

1
10
100
1000
10000
100000
1000000
10000000
100000000


###generate a CSV (comma seperated values) file for the recorded data, to be used in Graphlab Create

In [20]:
def generate_csv(name='default.csv',**kwargs) : 
    csv_file = open(name,'w')
    
    keys = kwargs.keys()
    values = kwargs.values()
    
    for key in keys[:-1] : 
        csv_file.write(key),
        csv_file.write(','),
    csv_file.write(keys[-1])
    csv_file.write('\n')

    for i in range(len(values[0])) : 
        for j in range(len(values)-1) : 
            csv_file.write(str(values[j][i])),
            csv_file.write(','),
        csv_file.write(str(values[len(values)-1][i]))
        csv_file.write('\n')
        
    csv_file.close()

In [21]:
generate_csv(name='test.csv',c_time_without_register = C_time_array_without_register_values,\
             c_time_with_register_values = C_time_array_with_register_values,\
             python_time = Python_time_array,count = count_array)
dataset = gl.SFrame('test.csv')
dataset

PROGRESS: Finished parsing file /Users/ironstein/Documents/projects working directory/handwriting recognition using neural networks on FPGA final year project/image processing/ipython notebooks/python C extension testing/mark3/test.csv
PROGRESS: Parsing completed. Parsed 9 lines in 0.010127 secs.
------------------------------------------------------
Inferred types from first line of file as 
column_type_hints=[int,float,float,float]
If parsing fails due to incorrect types, you can correct
the inferred type list above and pass it to read_csv in
the column_type_hints argument
------------------------------------------------------
PROGRESS: Finished parsing file /Users/ironstein/Documents/projects working directory/handwriting recognition using neural networks on FPGA final year project/image processing/ipython notebooks/python C extension testing/mark3/test.csv
PROGRESS: Parsing completed. Parsed 9 lines in 0.013473 secs.


count,python_time,c_time_with_register_values ...,c_time_without_register
1,0.00500679016113,0.000953674316406,0.018835067749
10,0.00309944152832,0.000953674316406,0.00405311584473
100,0.00715255737305,0.000953674316406,0.00500679016113
1000,0.0510215759277,0.0,0.00691413879395
10000,0.554084777832,0.00119209289551,0.0581741333008
100000,3.91387939453,0.00214576721191,0.291109085083
1000000,65.8710002899,0.000953674316406,2.33817100525
10000000,497.354984283,0.00190734863281,23.875951767
100000000,5080.3000927,0.00190734863281,234.472990036


In [22]:
gl.canvas.set_target('ipynb')
dataset.show(view='Line Chart',y='python_time',x='count')